In [28]:
import os
from dataclasses import dataclass
from pathlib import Path
import yaml

In [34]:
'''
Setting up the directories 
'''

CURRENT_DIRECTORY = os.getcwd()
PARENT_DIRECTORY = os.path.dirname(CURRENT_DIRECTORY)

# data Directory
DATA_PARENT_DIRECTORY = os.path.dirname(PARENT_DIRECTORY)
DATA_DIRECTORY = os.path.join(DATA_PARENT_DIRECTORY, '/_data_DL_Chicken_Disease_Classification_AWS_AZURE_DVC')

TRAIN_DATA = os.path.join(DATA_PARENT_DIRECTORY, '/train')
VALIDATION_DATA = os.path.join(DATA_PARENT_DIRECTORY, '/test')
TEST_DATA = os.path.join(DATA_PARENT_DIRECTORY, '/validation')

# print('CURRENT_DIRECTORY', CURRENT_DIRECTORY)
# print('PARENT_DIRECTORY', PARENT_DIRECTORY)
# print('DATA_PARENT_DIRECTORY', DATA_PARENT_DIRECTORY)
# print('DATA_DIRECTORY', DATA_DIRECTORY)
# print('TRAIN_DATA', TRAIN_DATA)
# print('VALIDATION_DATA', VALIDATION_DATA)
# print('TEST_DATA', TEST_DATA)

In [52]:
# 4. UPDATE the entity

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    train_dir : Path
    validation_dir : Path
    test_dir : Path


# Function to load config from YAML and create a DataIngestionConfig instance
def load_data_ingestion_config(config_path: str) -> DataIngestionConfig:
    with open(config_path, 'r') as file:
        config = yaml.safe_load(file)

    data_root_path = Path(config['data_root_path'])
    data_ingestion = config['data_ingestion']

    return DataIngestionConfig(
        root_dir = Path(data_ingestion['root_dir']),
        train_dir = data_root_path/data_ingestion['train_dir'],
        validation_dir = data_root_path/data_ingestion['validation_dir'],
        test_dir = data_root_path/data_ingestion['test_dir']
    )


# Usage 
CONFIG_DIRECTORY = os.path.join(PARENT_DIRECTORY, 'config')
config_path = os.path.join(CONFIG_DIRECTORY, 'config.yaml') 
data_ingestion_config = load_data_ingestion_config(config_path)
print(data_ingestion_config)

DataIngestionConfig(root_dir=WindowsPath('artifacts/data_ingestion'), train_dir=WindowsPath('C:/Users/petersunny.markappa/_00_Peter_Practice_Projects/_02_In_Progress/_02_ML_Projects_Krish_Naik_Course_list/02_DL_MLOPS_DVC_Azure_AWS/_data_DL_Chicken_Disease_Classification_AWS_AZURE_DVC/train'), validation_dir=WindowsPath('C:/Users/petersunny.markappa/_00_Peter_Practice_Projects/_02_In_Progress/_02_ML_Projects_Krish_Naik_Course_list/02_DL_MLOPS_DVC_Azure_AWS/_data_DL_Chicken_Disease_Classification_AWS_AZURE_DVC/validation'), test_dir=WindowsPath('C:/Users/petersunny.markappa/_00_Peter_Practice_Projects/_02_In_Progress/_02_ML_Projects_Krish_Naik_Course_list/02_DL_MLOPS_DVC_Azure_AWS/_data_DL_Chicken_Disease_Classification_AWS_AZURE_DVC/test'))


In [49]:
from ImageClassification.constants import *
from ImageClassification.utils.common import read_yaml, create_directories

In [51]:
# 5. UPDATE the configuration manager in src config
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            train_dir = config.train_dir,
            validation_dir = config.validation_dir,
            test_dir = config.test_dir
        )
        return data_ingestion_config
        
        

In [53]:
# 6. UPDATE the components
import os
import urllib.request as request
import zipfile
from ImageClassification import logger
from ImageClassification.utils.common import get_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            fi






In [ ]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)